In [91]:
#r "nuget: xunit, 2.8.1"
#r "nuget: ScottPlot, 4.1.29"

using System;
using System.Threading;
using System.Linq;
using Xunit;
using ScottPlot;
using System.Diagnostics;

class DefiniteIntegral 
{ 
    public static double OneThread(double a, double b, Func<double, double> function, double step) 
    { 
        double result = 0.0;
        for (double i = a; i < b; i += step)
        {
            result += (function(i) + function(i + step)) * step / 2;
        }
        return result; 
    } 

    public static double ManyThreads(double a, double b, Func<double, double> function, double step, int threadsnumber) 
    { 
        long resultat = 0; 
        Barrier barrier = new Barrier(threadsnumber);     
        Thread[] threads = Enumerable.Range(0, threadsnumber).Select(index => 
        { 
            Thread thread = new Thread(() => 
            { 
                double tResultat = 0.0; 
                double start = a + index * ((b - a) / threadsnumber); 
                double end = start + ((b - a) / threadsnumber); 

                for (double i = start; i < end; i += step) 
                { 
                    tResultat += (function(i) + function(i + step)) * step * 0.5; 
                } 

                Interlocked.Add(ref resultat, (long)(tResultat * 1e5)); 
            }); 

            thread.Start(); 
            return thread; 
        }).ToArray(); 

        foreach (Thread thread in threads) 
        { 
            thread.Join(); 
        }

        return resultat / 1e5; 
    } 
}

Installed Packages ScottPlot, 4.1.29 xunit, 2.8.1

In [92]:
(double resultat, double PrTime) Time(Func<double, double, Func<double, double>, double, double> func, double a, double b, Func<double, double> function, double step)
{
    var stopwatch = Stopwatch.StartNew();
    double resultat = func(a, b, function, step);
    stopwatch.Stop();
    return (resultat, stopwatch.Elapsed.TotalSeconds);
}

In [93]:
double[] steps = {1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6};
double accuracy = 1e-4;
Func<double, double> function = Math.Sin;

double ostep = steps[0];

foreach (var step in steps)
{
    var (integral, PrTime) = Time(DefiniteIntegral.OneThread, -100.00, 100.00, function, step);
    Console.WriteLine($"Шаг = {step} ▶️ интеграл = {integral} ▶️ время = {PrTime} с ▶️ точность = {Math.Abs(integral):F4}");
    
    if (Math.Abs(integral) <= accuracy)
    {
        ostep = step;
        Console.WriteLine($"Оптимальный размер шага ✅ {ostep}");
        break;
    }
}

Шаг = 0.1 ▶️ интеграл = -0.04620566619422959 ▶️ время = 0.000217 с ▶️ точность = 0.0462
Шаг = 0.01 ▶️ интеграл = -1.4438805186200998E-11 ▶️ время = 0.0006245 с ▶️ точность = 0.0000
Оптимальный размер шага ✅ 0.01


In [94]:
int[] kolThreads = { 2, 3, 4, 5, 6, 7, 8, 9 };
var ManyThreadsTimes = new List<double>();
foreach (var num in kolThreads)
{
    var (_, PrTime) = Time((a, b, func, step) => DefiniteIntegral.ManyThreads(a, b, func, step, num), -100.00, 100.00, function, ostep);
    ManyThreadsTimes.Add(PrTime);
    Console.WriteLine($"Количество потоков = {num} ▶️ время ⏱ = {PrTime} секунды");
}

Количество потоков = 2 ▶️ время ⏱ = 0.0020138 секунды
Количество потоков = 3 ▶️ время ⏱ = 0.0010884 секунды
Количество потоков = 4 ▶️ время ⏱ = 0.0013911 секунды
Количество потоков = 5 ▶️ время ⏱ = 0.0013079 секунды
Количество потоков = 6 ▶️ время ⏱ = 0.0016309 секунды
Количество потоков = 7 ▶️ время ⏱ = 0.0023487 секунды
Количество потоков = 8 ▶️ время ⏱ = 0.0018057 секунды
Количество потоков = 9 ▶️ время ⏱ = 0.0022172 секунды


In [95]:
var plot = new ScottPlot.Plot(700, 500);
plot.AddScatter(kolThreads.Select(n => (double)n).ToArray(), ManyThreadsTimes.ToArray());
plot.Title("Время выполнения в зависимости от количества потоков⏱");
plot.XLabel("Количество потоков");
plot.YLabel("Время выполнения в секундах⏱");
plot.SaveFig("graf.png");

In [96]:
double oManyThreadTime = ManyThreadsTimes.Min();
int oThreads = kolThreads[ManyThreadsTimes.IndexOf(oManyThreadTime)];
var (singleThreadIntegral, OneThreadTime) = Time(DefiniteIntegral.OneThread, -100.00, 100.00, function, ostep);
Console.WriteLine($"Время выполнения в однопоточном режиме  {OneThreadTime:F4} секунд ⏱");
Console.WriteLine($"Время выполнения в многопоточном режиме с {oThreads} потоками: {oManyThreadTime:F4} секунд ⏱");
Console.WriteLine($"Ускорение: {OneThreadTime/oManyThreadTime:F2}");
Console.WriteLine($"Увеличение производительности: {(OneThreadTime - oManyThreadTime) / OneThreadTime * 100:F2}%");

string itog = $@"
Итог:
👣 Оптимальный размер шага = {ostep}
🔁 Оптимальное количество потоков = {oThreads}
⏱ Время выполнения в однопоточном режиме = {OneThreadTime:F4} секунд
⏱ Время выполнения в многопоточном режиме с {oThreads} потоками = {oManyThreadTime:F4} секунд
💯 Ускорение = {OneThreadTime / oManyThreadTime:F2}x
📈 Увеличение производительности = {(-(OneThreadTime - oManyThreadTime)) / OneThreadTime  * 100 :F2}%
";
Console.WriteLine(itog);

Время выполнения в однопоточном режиме  0.0006 секунд ⏱
Время выполнения в многопоточном режиме с 3 потоками: 0.0011 секунд ⏱
Ускорение: 0.53
Увеличение производительности: -88.60%

Итог:
👣 Оптимальный размер шага = 0.01
🔁 Оптимальное количество потоков = 3
⏱ Время выполнения в однопоточном режиме = 0.0006 секунд
⏱ Время выполнения в многопоточном режиме с 3 потоками = 0.0011 секунд
💯 Ускорение = 0.53x
📈 Увеличение производительности = 88.60%

